### Model Training
## 1.1 Import Data and Required Packages
* Importing Pandas, Numpy, Matplotlib, Seaborn and Warings Library.

In [1]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
# from xgboost import XGBRegressor
import warnings

### Importing the data

In [2]:
df = pd.read_csv('data/stud.csv')
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


### Independent and dependent feature creation

In [3]:
X = df.drop(columns=['math score'], axis=1)
X.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75


In [4]:
y = df['math score']
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math score, Length: 1000, dtype: int64

### Column transformer creation

In [5]:
num_features = X.select_dtypes(exclude='object').columns
cat_features = X.select_dtypes(include='object').columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

std_transformer = StandardScaler()
onehot_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ('OneHotEncoder', onehot_transformer, cat_features),
        ('StandardScaler', std_transformer, num_features)
    ]
)


In [6]:
X = preprocessor.fit_transform(X)

In [7]:
X.shape

(1000, 19)

### Train-test split

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape, X_test.shape

((800, 19), (200, 19))

### Method to evaluate the model

In [9]:
def evaluate_model(true_val, actual_val):
    mae = mean_absolute_error(true_val, actual_val)
    mse = mean_squared_error(true_val, actual_val)
    rmse = np.sqrt(mse)
    r2score = r2_score(true_val, actual_val)
    return mae, mse, rmse, r2score

### Train the models

In [11]:
models = {
    "Linear Regression": LinearRegression(),
    "Ridge": Ridge(),
    "Lasso": Lasso(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_list = []
r2_list =[]

### Model training
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    ### Training and test accuracy
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    model_train_mae, model_train_mse, model_train_rmse, model_train_r2score = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_mse, model_test_rmse, model_test_r2score = evaluate_model(y_test, y_test_pred)

    print(list(models.keys())[i])
    print("-"*35)
    model_list.append(list(models.keys())[i])


    print("MODEL TRAINING PERFORMANCE TEST")
    print("mean absolute error: {:.2f}".format(model_train_mae))
    print("r_2score: {:.2f}".format(model_train_r2score))
    print("-"*30)

    print("MODEL TEST PERFORMANCE TEST")
    print("mean absolute error: {:.2f}".format(model_test_mae))
    print("r_2score: {:.2f}".format(model_test_r2score))
    r2_list.append(model_test_r2score)
    print("="*30)
    print("\n")

Linear Regression
-----------------------------------
MODEL TRAINING PERFORMANCE TEST
mean absolute error: 4.27
r_2score: 0.87
------------------------------
MODEL TEST PERFORMANCE TEST
mean absolute error: 4.23
r_2score: 0.88


Ridge
-----------------------------------
MODEL TRAINING PERFORMANCE TEST
mean absolute error: 4.26
r_2score: 0.87
------------------------------
MODEL TEST PERFORMANCE TEST
mean absolute error: 4.21
r_2score: 0.88


Lasso
-----------------------------------
MODEL TRAINING PERFORMANCE TEST
mean absolute error: 5.21
r_2score: 0.81
------------------------------
MODEL TEST PERFORMANCE TEST
mean absolute error: 5.16
r_2score: 0.83


K-Neighbors Regressor
-----------------------------------
MODEL TRAINING PERFORMANCE TEST
mean absolute error: 4.52
r_2score: 0.86
------------------------------
MODEL TEST PERFORMANCE TEST
mean absolute error: 5.64
r_2score: 0.78


Decision Tree
-----------------------------------
MODEL TRAINING PERFORMANCE TEST
mean absolute error: 0

In [12]:
pd.DataFrame(list(zip(model_list, r2_list)), columns=['Model Name', 'R2_Score']).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
1,Ridge,0.880593
0,Linear Regression,0.879216
5,Random Forest Regressor,0.853276
6,CatBoosting Regressor,0.851632
7,AdaBoost Regressor,0.842814
2,Lasso,0.825320
3,K-Neighbors Regressor,0.783497
4,Decision Tree,0.723184


### Difference between predicted and actual value

In [15]:
pred_df = pd.DataFrame({'Actual value: ': y_test, 'Predicted value: ': y_test_pred, 'Difference': y_test - y_test_pred})
pred_df.head()

,Actual value:,Predicted value:,Difference
521,91,76.235043,14.764957
737,53,57.027972,-4.027972
740,80,77.664062,2.335938
660,74,77.368984,-3.368984
411,84,83.205479,0.794521
